# 1. Подготовка

Импортируем библиотеки.

In [1]:
import requests
import pathlib
import client_lib # импортируем библиотеку для работы с ML Space

Устанавливаем переменные.

In [2]:
BASE_DIR = str(pathlib.Path().absolute())
print(f"Working dir: {BASE_DIR}")

Working dir: /home/jovyan/aicloud-examples/quick-start/job_launch_pt


# 2. Запуск задачи обучения

Запускать задачу в регионе будем с помощью класса `client_lib.Job()`.

Обязательные параметры для запуска задачи обучения:
- `script` – путь к запускаемому скрипту;
- `base_image` – базовый образ, в котором будет исполняться скрипт обучения модели;
- `instance_type` – конфигурация вычислительных ресурсов, используемых для решения задач.

Подробное описание параметров – в [документации client_lib](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/client-lib__job.html).

По умолчанию задачи запускаются в регионе Christofari.V100. Чтобы запустить задачу в другом регионе, необходимо указать его в параметре `region`. 

См. [Список регионов и их обозначения в client_lib](https://cloud.ru/docs/aicloud/mlspace/concepts/client-lib__job.html#row-body-region).

Для примера запустим задачу в регионе Cloud.Region.A100, обозначенном как `SR002-MT`.

Для масштабирования задачи доступны следующие параметры:

- `n_workers` – количество рабочих узлов региона, на котором будет исполняться скрипт;
- `instance_type` – конфигурация вычислительных ресурсов, используемых для решения задач.

Для выбора значения параметра `instance_type` воспользуемся методом `get_instance_types()`.

Выведем значения `instance_type` и посмотрим, какие доступны для региона `SR002-MT`.

In [3]:
client_lib.get_instance_types(regions="SR002-MT")

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━┳━━━━━┓
┃ region   ┃ instance_type           ┃ memory ┃ cpu ┃ gpu ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━╇━━━━━┩
│ SR002-MT │ a100.1gpu.40            │ 230Gi  │ 12  │ 1   │
│ SR002-MT │ a100.2gpu.40            │ 460Gi  │ 24  │ 2   │
│ SR002-MT │ a100.3gpu.40            │ 690Gi  │ 36  │ 3   │
│ SR002-MT │ a100.4gpu.40            │ 920Gi  │ 48  │ 4   │
│ SR002-MT │ a100.1gpu.80vG.12C.96G  │ 80Gi   │ 10  │ 1   │
│ SR002-MT │ a100.2gpu.80vG.24C.192G │ 160Gi  │ 20  │ 2   │
│ SR002-MT │ a100.3gpu.80vG.36C.288G │ 240Gi  │ 30  │ 3   │
│ SR002-MT │ a100.4gpu.80vG.48C.384G │ 320Gi  │ 40  │ 4   │
│ SR002-MT │ a100.5gpu.80vG.60C.480G │ 400Gi  │ 50  │ 5   │
│ SR002-MT │ a100.6gpu.80vG.72C.576G │ 480Gi  │ 60  │ 6   │
│ SR002-MT │ a100.7gpu.80vG.84C.672G │ 560Gi  │ 70  │ 7   │
│ SR002-MT │ a100.8gpu.80vG.96C.768G │ 640Gi  │ 80  │ 8   │
└──────────┴─────────────────────────┴────────┴─────┴─────┘

Для примера запустим задачу на 1 воркере с 1 GPU.

Сохраним в переменные название региона, `instanse_type` и образа.

In [4]:
REGION = "SR002-MT"
INSTANCE_TYPE = "a100.1gpu.40"
N_WORKERS = 1
BASE_IMAGE = "cr.ai.cloud.ru/aicloud-base-images/cuda12.1-torch2-py39:0.0.36"

In [5]:
job = client_lib.Job(
    base_image=BASE_IMAGE,
    script=f"{BASE_DIR}/train_distributed_example-torch2.py",
    region=REGION,
    instance_type=INSTANCE_TYPE,
    n_workers=N_WORKERS,
    type="pytorch2",
    processes_per_worker=1,
    job_desc="pytorch2 | client_lib | use_env=False | torch2",
)


Запустим задачу методом `submit()`.

In [6]:
job.submit()

'Job "lm-mpi-job-53fda410-5482-48b3-badf-c7f1b21135b0" created.'

Для получения статуса задачи воспользуемся методом `status()`.

См. [Список статусов задачи](https://cloud.ru/docs/aicloud/mlspace/concepts/guides/guides__env__common/environments__model-training__overview-statistics.html).

In [7]:
job.status()

'Job status=Pending'

Для просмотра логов задачи вызовем метод `logs()`.

Логи будут доступны после запуска задачи, то есть ее перехода в статус "Running".

In [8]:
job.logs()

"Job lm-mpi-job-53fda410-5482-48b3-badf-c7f1b21135b0 in queue. Try later"


Задача завершится автоматически после выполнения скрипта. Если необходимо прервать выполнение задачи, используется методом `kill()`.

In [9]:
job.kill()

'Job "lm-mpi-job-53fda410-5482-48b3-badf-c7f1b21135b0" deleted'

# 3. [Опционально] Сохранение промежуточных результатов обучения модели

Если в процессе обучения модели вы сохраняете промежуточные результаты, или чекпоинты, обучения, они попадают в папку `./logs`. Их можно скачать через веб-интерфейс Jupyter-ноутбука или скопировать из локально доступной файловой системы в хранилище S3.

## Выгрузка результатов обучения модели с NFS на S3

Для переноса файлов между NFS и S3 можно использовать [методы копирования client_lib](![image.png](attachment:image.png)) или [правила переноса Data Transfer Service](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/guides/guides__dc/data-catalog__data-processing__create-transfer-rule.html)

Рассмотрим копирование файлов из NFS на S3 воркспейса с помощью метода ```copy_from_nfs()```.



In [24]:
relative_path = str(pathlib.Path().absolute().relative_to('/home/jovyan'))

client_lib.copy_from_nfs(
    source_path=f"{relative_path}/logs/", # укажем путь к папке logs без /home/jovyam 
    from_region=client_lib.RegionEnum.SR002_MT, # укажем регион, в нашем случае SR002-MT
    destination_path="quck-start" # укажем место назначения переноса
)

TransferData(id='a58d680a-e927-4320-9e6a-da988e88f740', created_transfer_data={'cluster_name': 'pd11', 'connector_id': '04930312-40ed-42d1-8d3a-ac510b0393d0', 'created': '2024-05-23T12:51:03.464797', 'description': 'Copy data from NFS. Called from jupyter client_lib', 'destination_connector_id': '0a30997d-b205-459e-9179-89d36a2ed0cf', 'destination_source_category': 's3mlspace', 'execution_date': None, 'modified': '2024-05-23T12:51:03.464797', 'name': 'from_nfs-c299f91f-b732-43dc-84c2-eeace305978c-04930312-40ed-42d1-8d3a-ac510b0393d0', 'query': {'destination': 'quck-start', 'source': ['aicloud-examples/quick-start/job_launch_pt/logs']}, 'source_category': 'nfs', 'strategy': 'write_all', 'system': False, 'transfer_id': 'a58d680a-e927-4320-9e6a-da988e88f740', 'user_id': '4439ac3b-035e-490a-8df6-aba5b32a7695', 'workspace_id': '04930312-40ed-42d1-8d3a-ac510b0393d0'})

С помощью ID посмотрим логи переноса.

In [ ]:
client_lib.get_transfer_data_logs("3ebe26ec-1d6f-4797-926e-642f1fcbe12f") # id переноса берём из вывода предыдущей ячейки

В результате мы перенесли папку logs в S3 воркспейса в папку quick-start.

Чтобы проверить наличие файлов на S3, необходимо в разделе "Data Catalog" -> "Объектное Хранилище" выбрать бакет воркспейса с публичным доступом.